In [1]:
import pandas as pd
import os
import tqdm
from sklearn.preprocessing import OneHotEncoder
import utils.featurecreations as featurecreations
import importlib
import torch
import torch.nn as nn
from goto_conversion import goto_conversion

In [2]:
# dfs = []
# for file in tqdm.tqdm(os.listdir('data')):
#     try:
#         df = pd.read_excel(f'data/{file}')
#         dfs.append(df)
#     except:
#         print(file,'failed')

In [3]:
# data = pd.concat(dfs)

In [4]:
# data = data.reset_index(drop=True)

In [5]:
# data.to_feather('all_races_data.fth')

In [6]:
df = pd.read_feather('all_races_data.fth')

In [7]:
data = df

In [8]:
data.shape

(566575, 89)

In [9]:
data['meetingId'] = data['meetingId'].astype(int).astype(str)
data['raceId'] = data['raceId'].astype(int).astype(str)
data['horseId'] = data['horseId'].astype(int).astype(str)

In [10]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting', 'isBarrierTrial',
       'hasSectionals', 'resulted', 'raceNo', 'raceName', 'prizeMoney',
       'starters', 'startTime', 'class', 'distance', 'prizeMoneyBreakDown',
       'ageRestrictions', 'SexRestrictions', 'jockeyRestrictions',
       'trackCondition', 'trackConditionNumber', 'raceTime', 'sectional',
       'sectionalDistance', 'PFRaceTime', 'PFRaceL600Time', 'limitWeight',
       'weightType', 'jumps', 'groupStatus', 'tabNo', 'position', 'margin',
       'horse', 'weight', 'barrier', 'inRun', 'flucs', 'priceSP', 'stewards',
       'priceBF', 'gears', 'sex', 'age', 'CareerStarts', 'sire', 'dam',
       'foalDate', 'trainer', 'jockey', 'jockeyId', 'trainerId', 'jock_claim',
       'Reliable', 'RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       'RunStyle', 'Publish

In [11]:
data.priceSP.value_counts()

priceSP
0.00      93342
21.00     19765
31.00     18641
26.00     17851
101.00    14198
          ...  
1.05          1
124.60        1
268.00        1
226.20        1
121.10        1
Name: count, Length: 1479, dtype: int64

In [12]:
data.priceBF.value_counts()

priceBF
0.000000      215078
55.000000       2348
12.000000       2147
10.500000       2113
11.000000       2091
               ...  
115.908981         1
7.705373           1
52.731251          1
5.273125           1
70.274529          1
Name: count, Length: 223825, dtype: int64

In [13]:
race1 = data[data['raceId']=='879736']

In [14]:
race1

,meetingId,raceId,horseId,meetingDate,track,trackAbbrev,State,MPC,railPosition,TABMeeting,...,Pos600,Pos200,Marg600,Marg200,MargFin,Last600Time,Last200Time,ABML600,ABML200,ABMFin
1167,178543,879736,921720,03/09/2021,Albury,ALB,NSW,C,True,True,...,5,3,5.1,1.3,0.0,35.20,12.50,-3.5,0.8,5.4
1168,178543,879736,909170,03/09/2021,Albury,ALB,NSW,C,True,True,...,6,4,5.3,2.5,3.7,35.84,12.96,0.7,3.8,9.8
1169,178543,879736,941305,03/09/2021,Albury,ALB,NSW,C,True,True,...,4,2,3.6,1.1,0.5,35.52,12.60,-1.1,1.7,6.0
1170,178543,879736,884221,03/09/2021,Albury,ALB,NSW,C,True,True,...,3,5,2.5,3.4,6.7,36.86,13.36,7.6,6.3,13.4
1171,178543,879736,917728,03/09/2021,Albury,ALB,NSW,C,True,True,...,2,6,2.4,4.3,11.5,37.68,14.00,13.5,11.0,19.2
1172,178543,879736,893930,03/09/2021,Albury,ALB,NSW,C,True,True,...,1,1,0.0,0.0,3.5,36.68,13.36,6.8,6.6,9.6


In [15]:
# data['isBarrierTrial'].value_counts()

In [16]:
# data['stats_sum'] = data[stats_cols].fillna(0).sum(axis='columns' )

In [17]:
# data['stats_sum']

In [18]:
# data['stats_sum'].value_counts()

In [2]:
stats_cols = ['RPR', 'RPP', 'WP20R', 'WP20P', 'TP20R', 'TP20P', 'WPTP20R',
       'WPTP20P', 'RSFI', 'ScRank', 'SPrice', 'PFScore', 'PrSe', 'AHSP',
       'ClsDiff','ETRPrRk','ETRPrice'
       ]

In [3]:
st = ''
for r in stats_cols:
    st = st+'+'+r

In [4]:
st

'+RPR+RPP+WP20R+WP20P+TP20R+TP20P+WPTP20R+WPTP20P+RSFI+ScRank+SPrice+PFScore+PrSe+AHSP+ClsDiff+ETRPrRk+ETRPrice'

In [20]:
data['dog_id'] = data['horseId']
data['box'] = data['barrier']
data['bfSP'] = data['priceSP']
data['margin'] = data['MargFin']
data['StartPrice'] = data['priceSP']
data['place'] = data['position']
data['runtime'] = data['ITimeToFin']
data['date'] = pd.to_datetime(data['meetingDate'], format='mixed', dayfirst=True ).dt.date
# data['date'] = pd.Timestamp(data['date'])
data['state'] = data['State']
data['dog_name'] = data['horse']

In [21]:
stats = data[stats_cols]

In [22]:
data.RunStyle.head()

0      op/mf
1      op/mf
2    no data
3      mf/bm
4      mf/bm
Name: RunStyle, dtype: object

In [23]:
data.RunStyle.value_counts()

RunStyle
mf         105407
bm         102434
op          72480
op/mf       67431
mf/bm       64248
l           31448
no data     28344
Name: count, dtype: int64

In [24]:
data['goto'] = 0
race_groups = data.groupby('raceId')

In [25]:
data

,meetingId,raceId,horseId,meetingDate,track,trackAbbrev,State,MPC,railPosition,TABMeeting,...,dog_id,box,bfSP,StartPrice,place,runtime,date,state,dog_name,goto
0,178515,879507,924156,01/09/2021,Ashburton,ASH,NZ,X,True,True,...,924156,2,13.6,13.6,5,0.0,2021-09-01,NZ,Maxwell,0
1,178515,879507,945590,01/09/2021,Ashburton,ASH,NZ,X,True,True,...,945590,8,9.6,9.6,9,0.0,2021-09-01,NZ,Seraphic,0
2,178515,879507,956462,01/09/2021,Ashburton,ASH,NZ,X,True,True,...,956462,1,26.0,26.0,10,0.0,2021-09-01,NZ,Cavalryman,0
3,178515,879507,941443,01/09/2021,Ashburton,ASH,NZ,X,True,True,...,941443,6,1.8,1.8,1,0.0,2021-09-01,NZ,Lovingyourwork,0
4,178515,879507,928793,01/09/2021,Ashburton,ASH,NZ,X,True,True,...,928793,9,6.7,6.7,6,0.0,2021-09-01,NZ,Choice Moment,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566570,199884,1030736,978927,21/10/2023,Caulfield,CAUL,VIC,M,Out 3m Entire Circuit,True,...,978927,11,8.5,8.5,0,0.0,2023-10-21,VIC,Saltaire,0
566571,199884,1030736,950861,21/10/2023,Caulfield,CAUL,VIC,M,Out 3m Entire Circuit,True,...,950861,4,21.0,21.0,0,0.0,2023-10-21,VIC,Willinga Beast,0
566572,199884,1030736,945687,21/10/2023,Caulfield,CAUL,VIC,M,Out 3m Entire Circuit,True,...,945687,7,14.0,14.0,0,0.0,2023-10-21,VIC,Miss Middle Park,0
566573,199884,1030736,954619,21/10/2023,Caulfield,CAUL,VIC,M,Out 3m Entire Circuit,True,...,954619,5,12.0,12.0,0,0.0,2023-10-21,VIC,Lempicka,0


In [26]:
for key,datag in tqdm.tqdm(race_groups):
    # print(key,data)
    num_horses = datag.shape[0]
    # print(num_horses)
    try:
        data['goto'].loc[datag.index[0]:datag.index[-1]] = goto_conversion(datag.priceSP)
    except Exception as e :
        pass
        # print(e)
    # break

  0%|          | 1/63000 [00:00<4:13:16,  4.15it/s]C:\Users\Nick\AppData\Local\Temp\ipykernel_11580\2082423460.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['goto'].loc[datag.index[0]:datag.index[-1]] = goto_conversion(datag.priceSP)
C:\Users\Nick\AppData\Local\Temp\ipykernel_11580\2082423460.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.48184915842726933, 1e-06, 0.013290850006421589, 0.38841573800635115, 1e-06, 1e-06, 0.022568506334205925, 0.08690998901413688, 0.027647047108710573]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data['goto'].loc[datag.index[0]:datag.index[-1]] = goto_conversion(datag.priceSP)
100%|██████████| 63000/63000 [00:10<00:00, 6208.59it/s]


In [27]:
data.goto.sum()

48889.71699137148

In [28]:
run_style_ohe = OneHotEncoder(sparse_output=False)
run_style_one_hot = run_style_ohe.fit_transform(data[['RunStyle']])
track_ohe = OneHotEncoder(sparse_output=False)
track_one_hot = track_ohe.fit_transform(data[['track']])

In [29]:
data["run_style_one_hot"] = run_style_one_hot.tolist()
data["tracks_one_hot"] = track_one_hot.tolist()

In [30]:
len(data["tracks_one_hot"].iloc[0])

404

In [31]:
horse_form = data.groupby(['horseId'], sort=False, as_index=False)

In [32]:
importlib.reload(featurecreations)

<module 'utils.featurecreations' from 'c:\\Users\\Nick\\Documents\\GitHub\\BetfairSpringCarnival\\src\\utils\\featurecreations.py'>

In [33]:
data = featurecreations.generate_prev_race(data, horse_form)

In [34]:
data['prev_race'].iloc[-1]

'1027248'

In [39]:
for k,v in tqdm(horse_groups:
    try:
        v.distance
    except Exception as e:
        print(e)

In [44]:
horse_groups = data.groupby('horseId', sort=False, as_index=False)
data = featurecreations.generate_horse_stats_simple(data,horse_groups,rolling_window=1)

In [41]:
data['dog_id'] = data['horseId']
data['box'] = data['barrier']
data['bfSP'] = data['priceSP']
data['margin'] = data['MargFin']
data['StartPrice'] = data['priceSP']
data['place'] = data['position']
data['runtime'] = data['ITimeToFin']
data['date'] = pd.to_datetime(data['meetingDate'], format='mixed', dayfirst=True ).dt.date
# data['date'] = pd.Timestamp(data['date'])
data['state'] = data['State']
data['dog_name'] = data['horse']
# data = data[]

In [43]:
# data.barrier?
data.columns
# data['State']

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting',
       ...
       'prev_race_track', 'prev_race_state', 'next_race', 'dist_last__10',
       'box_last__10', 'margin_fin_avg_10', 'margin_200_avg_10',
       'margin_600_avg_10', 'last_start_price', 'last_start_prob'],
      dtype='object', length=113)

In [45]:
stats_cols_new = stats_cols+['dist_last__10',
       'box_last__10', 'margin_fin_avg_10', 'margin_200_avg_10',
       'margin_600_avg_10', 'last_start_price', 'last_start_prob']

In [52]:
form_only_stats = data[stats_cols_new].fillna(-1.0)
stats_w_runstyle = [x+y for x,y in zip(form_only_stats.values.tolist(),data.run_style_one_hot)]
stats = pd.Series(stats_w_runstyle)

In [53]:
stats[0]

[7.0,
 16.81,
 3.0,
 7.89,
 3.0,
 7.18,
 3.0,
 7.52,
 0.0,
 25.0,
 900.0,
 70.0,
 1.0,
 41.3,
 0.3,
 25.0,
 900.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0]

In [51]:
x[0]

TypeError: 'zip' object is not subscriptable

In [54]:
# data = data.drop(columns=stats_cols)
data['stats'] = stats

In [55]:
data = data.query('barrier > 0 and barrier <=20')
data.barrier.value_counts()

barrier
2     63016
1     63009
3     62932
4     62355
5     60389
6     56511
7     50664
8     43393
9     35033
10    26904
11    18472
12    12456
13     6137
14     3659
15      935
16      511
17      101
18       68
19       14
20        9
Name: count, dtype: int64

In [56]:
data['box'] = data['barrier']

In [57]:
data['runtime'] = data['ITimeToFin']
data['race_grade'] = data['class']

In [58]:
data['stats_cols'] = str([stats_cols])
data['stats_cols'] = data['stats_cols'].astype('category')

In [59]:
data = data[~data['isBarrierTrial']]

In [60]:
data = data.reset_index(drop=True)

In [61]:
data.shape

(473226, 121)

In [62]:
race = data[data['raceId']=='879552']

In [63]:
race.barrier

504    7
505    5
506    6
507    2
508    1
509    8
510    4
511    3
Name: barrier, dtype: int64

In [64]:
data.goto.mean()

0.10331110705061572

In [65]:
empty_margin_list = [100]*20
margin_list = [x for x in race["margin"]]
boxes_list = [int(x) for x in race['box']]
for n,x in enumerate(boxes_list):
    empty_margin_list[x-1] = margin_list[n]

In [66]:
print(empty_margin_list)

[0.0, 1.9, 0.2, 1.3, 4.8, 3.9, 1.8, 0.1, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]


In [67]:
def boosted_softmin(input):
    sft_min = nn.Softmin(dim=-1)
    return sft_min(torch.exp(input))

In [68]:
adjustedMargin = (boosted_softmin(torch.tensor(empty_margin_list)))

In [69]:
adjustedMargin

tensor([3.5963e-01, 1.2204e-03, 2.8820e-01, 2.4922e-02, 0.0000e+00, 3.4271e-22,
        2.3058e-03, 3.2372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00])

In [70]:
adjustedMargin.to('cuda:0')

tensor([3.5963e-01, 1.2204e-03, 2.8820e-01, 2.4922e-02, 0.0000e+00, 3.4271e-22,
        2.3058e-03, 3.2372e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0')

In [71]:
data.reset_index(drop=True).to_feather('horsey_racing_v1.fth')

In [72]:
# data = data[data['raceId']=='891328']

In [73]:
print(data[['place','barrier']])

        place  barrier
0           5        2
1           9        8
2          10        1
3           1        6
4           6        9
...       ...      ...
473221      0       11
473222      0        4
473223      0        7
473224      0        5
473225      0        9

[473226 rows x 2 columns]


In [74]:
print(data.shape)
# data = data.

(473226, 121)


In [75]:
data.columns

Index(['meetingId', 'raceId', 'horseId', 'meetingDate', 'track', 'trackAbbrev',
       'State', 'MPC', 'railPosition', 'TABMeeting',
       ...
       'last_start_price', 'last_start_prob', 'dist_last__1', 'box_last__1',
       'margin_fin_avg_1', 'margin_200_avg_1', 'margin_600_avg_1', 'stats',
       'race_grade', 'stats_cols'],
      dtype='object', length=121)

In [76]:
x = data.groupby('track')['barrier'].max()

In [77]:
data.barrier.value_counts()

barrier
2     49030
1     49025
3     49017
4     48890
5     48290
6     46666
7     43525
8     38789
9     32450
10    25604
11    18157
12    12352
13     6134
14     3659
15      935
16      511
17      101
18       68
19       14
20        9
Name: count, dtype: int64

In [78]:
data.groupby('raceId').count().mean()

meetingId           9.652158
horseId             9.652158
meetingDate         9.652158
track               9.652158
trackAbbrev         9.652158
                      ...   
margin_200_avg_1    9.080893
margin_600_avg_1    9.080893
stats               9.652158
race_grade          9.652158
stats_cols          9.652158
Length: 120, dtype: float64